In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
import os
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge, Lasso, ElasticNet

import csv
import os
import json
import numpy as np
from datetime import datetime

c:\Users\franc\Desktop\MostoElMostro\MostoElMostro\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

folder = '../data/processed'
filename = 'dataset_final.csv'
file_path = os.path.join(folder, filename)

try:
    df = pd.read_csv(
        file_path,
        sep=',',
        decimal='.'
    )
    df = df.sort_values(by='DIA', ignore_index=True)
    print(f"✅ DataFrame cargado exitosamente desde: {file_path}")
    print(f"Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")

✅ DataFrame cargado exitosamente desde: ../data/processed\dataset_final.csv
Dimensiones: (1183, 107)

Primeras 5 filas:
          DIA  Frio (Kw)  Hl de Mosto  Sala Maq (Kw)  Servicios (Kw)  \
0  2020-07-01    14325.0          0.0        17080.0         23848.0   
1  2020-07-02    23954.0       2907.0        27216.0         38033.0   
2  2020-07-03    28268.0       4829.0        31386.0         42565.5   
3  2020-07-04    24246.0       7828.0        28070.0         39650.0   
4  2020-07-05    29885.0       6406.0        33463.0         45385.0   

   KW Gral Planta  Planta (Kw)  Agua Planta (Hl)  Planta de agua (Hl)  \
0         59058.0     27637.27           10280.0             11241.40   
1        131184.0     54409.81           13970.0             22107.77   
2        136078.0     65685.59           36300.0             46955.43   
3        139714.0     67098.54           40120.0             51124.18   
4        146862.0     70600.64           38940.0             49146.08   

   KW Tr

In [3]:

# 1️⃣ Cargar dataset definitivo
ruta_csv = "../data/processed/X_test_preproc.csv"
X_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/X_train_preproc.csv"
X_train = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_test.csv"
y_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_train.csv"
y_train = pd.read_csv(ruta_csv, sep=',', decimal='.')


print("Shape dataset:", df.shape)




Shape dataset: (1183, 107)


In [4]:
def objective_rf_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.1, 1.0, log=True)

    # Modelo
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_jobs=-1,
        random_state=42
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [5]:
def objective_xgb_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbosity=0
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [6]:
def objective_lgbm_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    num_leaves = trial.suggest_int("num_leaves", 20, 150)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [7]:
def objective_elasticnet_mae(trial):
    alpha = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0) 

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=2000)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

# 1. Crear los estudios (todos buscan minimizar el MAE)
study_rf_mae = optuna.create_study(direction="minimize")
study_xgb_mae = optuna.create_study(direction="minimize")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_elasticnet_mae = optuna.create_study(direction="minimize")

# 2. Ejecutar las optimizaciones
print("Optimizando Random Forest (MAE)...")
study_rf_mae.optimize(objective_rf_mae, n_trials=50)

print("Optimizando XGBoost (MAE)...")
study_xgb_mae.optimize(objective_xgb_mae, n_trials=75)

print("Optimizando LightGBM (MAE)...")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=75)

print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# 3. Ver los mejores parámetros
print(f"Mejor RF (MAE): {study_rf_mae.best_value} con params {study_rf_mae.best_params}")
print(f"Mejor XGB (MAE): {study_xgb_mae.best_value} con params {study_xgb_mae.best_params}")
print(f"Mejor LGBM (MAE): {study_lgbm_mae.best_value} con params {study_lgbm_mae.best_params}")
print(f"Mejor ElasticNet (MAE): {study_elasticnet_mae.best_value} con params {study_elasticnet_mae.best_params}")

In [8]:

def log_experiment(model_name: str, 
                   model_object: object, 
                   study: optuna.study.Study, 
                   X_train, y_train, X_test, y_test):
    """
    Entrena un modelo con los mejores parámetros de un estudio de Optuna,
    calcula métricas y las guarda en un CSV.
    """
    
    # --- 1. Definir la ruta del log ---
    log_dir = "results"
    log_file = os.path.join(log_dir, "experiment_logs.csv")
    os.makedirs(log_dir, exist_ok=True) # Crea la carpeta si no existe

    # --- 2. Obtener mejores parámetros y valor del estudio ---
    best_params = study.best_params
    best_value = study.best_value # El MAE de CV que encontró Optuna
    
    # --- 3. Instanciar y entrenar el modelo final ---
    # Manejo especial para modelos lineales que necesitan Pipeline
    if "pipeline" in model_name.lower():
        # El pipeline ya está definido en la función objective, 
        # aquí solo re-creamos el mejor.
        if model_name == "Pipeline_ElasticNet":
             # Re-crea el pipeline con los mejores params
             model =ElasticNet(
                    alpha=best_params.get('alpha'), 
                    l1_ratio=best_params.get('l1_ratio'),
                    random_state=42, 
                    max_iter=2000
                )
            
            
    else:
        # Para modelos no-pipeline (RF, XGB, LGBM)
        model = model_object(**best_params, random_state=42, n_jobs=-1)
        # Ajustes para modelos que no les gusta 'n_jobs' o necesitan 'verbose'
        if isinstance(model, (xgb.XGBRegressor, lgb.LGBMRegressor)):
            model.set_params(verbosity=0) if isinstance(model, xgb.XGBRegressor) else None
            model.set_params(verbose=-1) if isinstance(model, lgb.LGBMRegressor) else None

    # Entrenar el modelo final con todos los datos de train
    model.fit(X_train, y_train)

    # --- 4. Calcular Métricas (Train y Test) ---
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'optuna_cv_mae': best_value, # Métrica optimizada
        'mae_train': mean_absolute_error(y_train, y_pred_train),
        'mae_test': mean_absolute_error(y_test, y_pred_test),
        'rmse_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'rmse_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'r2_train': r2_score(y_train, y_pred_train),
        'r2_test': r2_score(y_test, y_pred_test),
        'hyperparameters': json.dumps(best_params) # Guardar params como string JSON
    }
    
    # --- 5. Escribir en el CSV ---
    file_exists = os.path.isfile(log_file)
    
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        
        if not file_exists:
            writer.writeheader() # Escribir cabecera solo si el archivo es nuevo
            
        writer.writerow(metrics)
        
    print(f"Resultados de '{model_name}' guardados en {log_file}")

In [9]:
# --- 1. Ejecutar el estudio (ejemplo con Random Forest) ---
print("Optimizando Random Forest (MAE)...")
study_rf_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_rf_mae' está definida)
study_rf_mae.optimize(objective_rf_mae, n_trials=15) 

# --- 2. Registrar el resultado ---
log_experiment(
    model_name="RandomForest",
    model_object=RandomForestRegressor, # Pasa la clase del modelo
    study=study_rf_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 3. Ejecutar el estudio (ejemplo con XGBoost) ---
print("Optimizando XGBoost (MAE)...")
study_xgb_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_xgb_mae' está definida)
study_xgb_mae.optimize(objective_xgb_mae, n_trials=15)

# --- 4. Registrar el resultado ---
log_experiment(
    model_name="XGBoost",
    model_object=xgb.XGBRegressor, # Pasa la clase del modelo
    study=study_xgb_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con ElasticNet) ---
print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_elasticnet_mae' está definida)
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=15)

# --- 6. Registrar el resultado ---
# Nota: 'model_object' es None porque se maneja dentro del 'if'
log_experiment(
    model_name="Pipeline_ElasticNet",
    model_object=None, 
    study=study_elasticnet_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# Repetir para LightGBM...

[I 2025-11-13 20:17:18,722] A new study created in memory with name: no-name-c402ec4b-3951-4ea9-bec9-98ed86a5b1d4


Optimizando Random Forest (MAE)...


[I 2025-11-13 20:17:26,935] Trial 0 finished with value: 68009.3376617101 and parameters: {'n_estimators': 200, 'max_depth': 27, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 0.17512321837013087}. Best is trial 0 with value: 68009.3376617101.
[I 2025-11-13 20:17:32,297] Trial 1 finished with value: 65756.35009985275 and parameters: {'n_estimators': 100, 'max_depth': 13, 'min_samples_split': 13, 'min_samples_leaf': 6, 'max_features': 0.8065810196908856}. Best is trial 1 with value: 65756.35009985275.
[I 2025-11-13 20:17:36,414] Trial 2 finished with value: 64778.22396586195 and parameters: {'n_estimators': 950, 'max_depth': 35, 'min_samples_split': 20, 'min_samples_leaf': 8, 'max_features': 0.1954827314142993}. Best is trial 2 with value: 64778.22396586195.
[I 2025-11-13 20:17:37,680] Trial 3 finished with value: 66063.9624720562 and parameters: {'n_estimators': 300, 'max_depth': 25, 'min_samples_split': 14, 'min_samples_leaf': 7, 'max_features': 0.1623177212316744}. Be

Resultados de 'RandomForest' guardados en results\experiment_logs.csv
Optimizando XGBoost (MAE)...


[I 2025-11-13 20:18:32,063] Trial 0 finished with value: 58050.63359375 and parameters: {'n_estimators': 300, 'learning_rate': 0.05103829963709337, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 0.00030609974707518904, 'reg_lambda': 3.177659753100075e-08}. Best is trial 0 with value: 58050.63359375.
[I 2025-11-13 20:19:13,288] Trial 1 finished with value: 62266.0078125 and parameters: {'n_estimators': 1500, 'learning_rate': 0.01599164344722994, 'max_depth': 9, 'subsample': 1.0, 'colsample_bytree': 0.9, 'reg_alpha': 0.00019729851604466834, 'reg_lambda': 0.00020280871858668857}. Best is trial 0 with value: 58050.63359375.
[I 2025-11-13 20:19:21,204] Trial 2 finished with value: 65068.8265625 and parameters: {'n_estimators': 400, 'learning_rate': 0.07084518170313625, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.8, 'reg_alpha': 5.653851477776792e-07, 'reg_lambda': 2.859738900015738e-06}. Best is trial 0 with value: 58050.63359375.
[I 2025-11-13 20:19:51,

Resultados de 'XGBoost' guardados en results\experiment_logs.csv
Optimizando ElasticNet (MAE)...


[I 2025-11-13 20:21:34,411] Trial 2 finished with value: 119544.8404353179 and parameters: {'alpha': 0.02528668473588323, 'l1_ratio': 0.5521020610225856}. Best is trial 1 with value: 84261.24053316866.
[I 2025-11-13 20:21:34,450] Trial 3 finished with value: 73606.31610029723 and parameters: {'alpha': 1.8047631903149544, 'l1_ratio': 0.3048255801205134}. Best is trial 3 with value: 73606.31610029723.
[I 2025-11-13 20:21:34,533] Trial 4 finished with value: 129497.78403168284 and parameters: {'alpha': 0.0036966243574217445, 'l1_ratio': 0.05498352891046454}. Best is trial 3 with value: 73606.31610029723.
[I 2025-11-13 20:21:34,572] Trial 5 finished with value: 80105.4476917838 and parameters: {'alpha': 2.840369735597952, 'l1_ratio': 0.8177312290159995}. Best is trial 3 with value: 73606.31610029723.
[I 2025-11-13 20:21:34,620] Trial 6 finished with value: 94216.26086279757 and parameters: {'alpha': 0.1865121655013306, 'l1_ratio': 0.46399797919167296}. Best is trial 3 with value: 73606.316

Resultados de 'Pipeline_ElasticNet' guardados en results\experiment_logs.csv
